# Advent of Code 2019 - Day 03

In [1]:
from pathlib import Path

INPUT = (Path().absolute() / "inputs" / "day03.txt").open().read()

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

## Day 3: Crossed Wires

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

    * R75,D30,R83,U83,L12,D49,R71,U7,L72
      U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
    * R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
      U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?

In [3]:
from collections import defaultdict
import itertools

PATH_DIRS = {
    "R": (1, 0),
    "L": (-1, 0),
    "U": (0, 1),
    "D": (0, -1)
}

def distance_to_intersection(wire1, wire2) -> int:
    visited = defaultdict(int)
    
    def visit_wire(wire):
        def expand_wire(wire):
            for direction, amount in ((x[0], x[1:]) for x in wire):
                for movement in itertools.repeat(PATH_DIRS[direction], int(amount)):
                    yield movement
                    
        current_x = 0
        current_y = 0
        wire_visited = set()
        for x, y in expand_wire(wire):
            current_x += x
            current_y += y
            pos = (current_x, current_y)
            if pos not in wire_visited:
                visited[pos] += 1
            wire_visited.add(pos)
            
    visit_wire(wire1)
    visit_wire(wire2)
    
    return min(abs(x) + abs(y) for (x, y), v in visited.items() if v == 2)

In [4]:
def solve_puzzle_part1(puzzle_input: str=INPUT) -> int:
    wires = [w.split(",") for w in puzzle_input.split("\n")]
    return distance_to_intersection(wires[0], wires[1])

In [5]:
solve_puzzle_part1(INPUT)

489

## Part Two

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

    * R75,D30,R83,U83,L12,D49,R71,U7,L72
      U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps
    * R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
      U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps

What is the fewest combined steps the wires must take to reach an intersection?

In [6]:
from collections import defaultdict
import itertools

PATH_DIRS = {
    "R": (1, 0),
    "L": (-1, 0),
    "U": (0, 1),
    "D": (0, -1)
}

def distance_to_intersection(wires) -> int:
    visited = defaultdict(lambda: defaultdict(int))
    
    for wire_idx, wire in enumerate(wires):
        def expand_wire(wire):
            for direction, amount in ((x[0], x[1:]) for x in wire):
                for movement in itertools.repeat(PATH_DIRS[direction], int(amount)):
                    yield movement
                    
        current_x = 0
        current_y = 0
        wire_visited = set()
        for steps, (x, y) in enumerate(expand_wire(wire), start=1):
            current_x += x
            current_y += y
            pos = (current_x, current_y)
            if pos not in wire_visited:
                visited[pos][wire_idx] = steps
            wire_visited.add(pos)
    
    return min(sum(v.values()) for (x, y), v in visited.items() if len(v) == 2)

In [7]:
def solve_puzzle_part2(puzzle_input: str=INPUT) -> int:
    wires = [w.split(",") for w in puzzle_input.split("\n")]
    return distance_to_intersection(wires)

In [8]:
solve_puzzle_part2(INPUT)

93654